In [8]:
import pandas as pd
import numpy as np
import joblib
import os
import json
import warnings
import tensorflow as tf
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential # Sequential 모델로 복구
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 경고 무시
warnings.filterwarnings('ignore')

# 1. 데이터 로드 및 전처리 파이프라인 준비
print("딥러닝 데이터 준비 중...")

# 경로 설정: 스크립트가 notebooks/ 폴더 안에 있다고 가정하고 상위 폴더를 참조
df = pd.read_csv('../data/spotify_churn_dataset.csv')

if 'user_id' in df.columns:
    df = df.drop(columns=['user_id'])

# Feature Engineering
df['ad_burden'] = df['ads_listened_per_week'] / (df['listening_time'] + 1)
df['satisfaction_score'] = df['songs_played_per_day'] * (1 - df['skip_rate'])
df['time_per_song'] = df['listening_time'] / (df['songs_played_per_day'] + 1)

X = df.drop(columns=['is_churned'])
y = df['is_churned']

# 3. 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 4. 전처리 Column 정의
numerical_cols = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate', 
    'ads_listened_per_week', 'offline_listening',
    'ad_burden', 'satisfaction_score', 'time_per_song'
]
categorical_cols = ['gender', 'country', 'subscription_type', 'device_type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# 5. BorderlineSMOTE 적용
print("BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)")
bsmote = BorderlineSMOTE(random_state=42, kind='borderline-1')
X_train_res, y_train_res = bsmote.fit_resample(X_train_processed, y_train)


# 6. 모델 설계 (Sequential 모델 - 최고 성능 구조 복구)
input_dim = X_train_res.shape[1]
# 최고 성능을 냈던 512-256-128-64-32 Sequential 구조로 복구
model = Sequential([
    Dense(512, input_dim=input_dim, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(256, activation='swish'),
    BatchNormalization(), Dropout(0.4),
    Dense(128, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(64, activation='swish'),
    BatchNormalization(), Dropout(0.3),
    Dense(32, activation='swish'),
    BatchNormalization(), Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
print(f"\n모델 구조: Sequential DNN (최고 성능 구조 복구) (입력 차원: {input_dim})")


# 7. 학습 시작
print("딥러닝 학습 시작...")
# 학습 안정화를 위한 파라미터 유지
EPOCHS = 250 
early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0)

checkpoint = ModelCheckpoint('../models/spotify_dl_model.h5', monitor='val_accuracy', save_best_only=True, verbose=0)

history = model.fit(
    X_train_res, y_train_res, epochs=EPOCHS, batch_size=64, validation_split=0.2,
    callbacks=[early_stop, reduce_lr, checkpoint], verbose=0
)


# 8. 최적의 임계값(Threshold) 찾기 및 저장
print("\n최적의 임계값 탐색 중...")
best_model = tf.keras.models.load_model('../models/spotify_dl_model.h5')
y_pred_proba = best_model.predict(X_test_processed).flatten()

thresholds = np.arange(0.3, 0.7, 0.01)
best_thresh = 0.5
best_f1 = 0
best_acc = 0

for thresh in thresholds:
    y_pred_temp = (y_pred_proba >= thresh).astype(int)
    f1_temp = f1_score(y_test, y_pred_temp)
    
    if f1_temp > best_f1:
        best_f1 = f1_temp
        best_acc = accuracy_score(y_test, y_pred_temp)
        best_thresh = thresh

# 최종 결과 출력
print("-" * 40)
print(f"🚀 딥러닝 최적 임계값 발견: {best_thresh:.2f}")
print(f"최종 Accuracy: {best_acc:.4f}")
print(f"최종 F1-Score: {best_f1:.4f}")
print("-" * 40)

# 9. 모델 및 메트릭 저장
if not os.path.exists('../models'): os.makedirs('../models')
joblib.dump(preprocessor, '../models/dl_preprocessor.pkl')

metrics_file = '../data/model_metrics.json'
final_metrics = {}

if os.path.exists(metrics_file):
    try:
        with open(metrics_file, 'r') as f:
            final_metrics = json.load(f)
    except:
        pass

# 메트릭 업데이트
final_metrics['Deep Learning (DNN)'] = {
    'Accuracy': best_acc, 
    'F1-Score': best_f1,
    'Best Threshold': float(best_thresh)
}

with open(metrics_file, 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("딥러닝 모델 및 점수 저장 완료.")

딥러닝 데이터 준비 중...
BorderlineSMOTE 적용 중... (시간이 조금 걸릴 수 있습니다)

모델 구조: Sequential DNN (최고 성능 구조 복구) (입력 차원: 20)
딥러닝 학습 시작...



최적의 임계값 탐색 중...
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
----------------------------------------
🚀 딥러닝 최적 임계값 발견: 0.52
최종 Accuracy: 0.8165
최종 F1-Score: 0.7425
----------------------------------------
딥러닝 모델 및 점수 저장 완료.
